[Source: Quantecon](https://python.quantecon.org/mle.html#top)

## Key Ideas
* likelihood functions are the distributions we believe generated the data we have.
* We are maximising the parameters of the distribution that best fit the data
* To maximise these functions we can Use the [Newton-Raphson algorithm](https://en.wikipedia.org/wiki/Newton%27s_method_in_optimization) or some other methods available in the `scipy.optimization` module like `minimize` [Source](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize)
* The key take away in Newton method is that we take the gradient and the Hessian of our probability distribution, in the case of the example, a Poisson distribution.
* We operate on the log of the pmf as it's easier to work with it afterwards when it comes to differentiation (take a look at Linear Regressions Basics)

In [1]:
import numpy as np
from scipy.special import factorial

class PoissonRegression:
    def __init__(self, X, y, beta):
        self.X = X
        n, k = X.shape
        self.y = y.reshape(n, 1)  # y as column vector.
        self.beta = beta.reshape(k, 1)  # match features dimension
        self.lbd = None  # init lambda
        self.update_lambda()

    def update_lambda(self):
        # this gives us the dot product of beta against each of the vectors in X
        # Why do we use this value for lambda? Well, it seems that if we remove the exp
        # the algorithm won't converge.
        self.lbd = np.exp(self.X @ self.beta)

    def get_log_likelihood(self):
        """Compute the log-likelihood distribution (poisson)."""
        a = self.y * np.log(self.lbd)  # ln(lbd^y), vector
        b = - np.log(factorial(self.y))  # ln(y!), vector
        c = - self.lbd  # ln(e^-lbd)
        return (a + b + c).sum()

    def get_gradient(self):
        """
        Compute the gradient of the log likelihood.

        Take a look at Joplin notes for the worked out logic from log_likelihood and this
        expression.
        """
        return self.X.T @ (self.y - self.lbd)

    def get_hessian(self):
        """
        Compute the Hessian matrix of the log likelihood.

        I didn't work out this out of the log likelihood but as it agrees with minimize
        algorithm so it should be right.
        """
        return -(self.X.T @ (self.lbd * self.X))

    def run(self):
        g, h = self.get_gradient(), self.get_hessian()
        new_beta = self.beta - (np.linalg.inv(h) @ g)  # update rule
        output = {"error": new_beta - self.beta}
        self.beta = new_beta
        self.update_lambda()  # as beta has changed
        output["beta"] = self.beta.ravel().tolist()
        output["log_likelihood"] = self.get_log_likelihood()
        return output


def newton_raphson(model, threshold=1e-3, max_iter=1000, display=True):
    i = 0
    error = 100

    if display:
        header = f'{"Iteration_k":<13}{"Log-likelihood":<16}{"θ":<60}'
        print(header)
        print("-" * len(header))

    while np.any(error > threshold) and i < max_iter:
        output = model.run()
        beta_list = [f'{t:.3}' for t in output["beta"]]
        if display:
            update = f'{i:<13}{output["log_likelihood"]:<16.8}{beta_list}'
            print(update)
        i += 1
        error = output["error"]
    print(f'Number of iterations: {i}')
    print(f'b_hat = {beta_list}')


X = np.array([[1, 2, 5],
              [1, 1, 3],
              [1, 4, 2],
              [1, 5, 2],
              [1, 3, 1]])

y = np.array([1, 0, 1, 1, 0])

# Take a guess at initial βs
init_beta = np.array([0.1, 0.1, 0.1])

# Create an object with Poisson model values
poi = PoissonRegression(X=X, y=y, beta=init_beta)

# Use newton_raphson to find the MLE
b_hat = newton_raphson(poi, display=True)


Iteration_k  Log-likelihood  θ                                                           
-----------------------------------------------------------------------------------------
0            -4.3447622      ['-1.49', '0.265', '0.244']
1            -3.5742413      ['-3.38', '0.528', '0.474']
2            -3.3999526      ['-5.06', '0.782', '0.702']
3            -3.3788646      ['-5.92', '0.909', '0.82']
4            -3.3783559      ['-6.07', '0.933', '0.843']
5            -3.3783555      ['-6.08', '0.933', '0.843']
Number of iterations: 6
b_hat = ['-6.08', '0.933', '0.843']


In [2]:
# Solution using scipy's optimize. Same, as expected
from scipy.optimize import minimize

X = np.array([[1, 2, 5],
              [1, 1, 3],
              [1, 4, 2],
              [1, 5, 2],
              [1, 3, 1]])

y = np.array([1, 0, 1, 1, 0])

# Take a guess at initial βs
init_beta = np.array([1, 1, 1])

def log_likelihood_poisson(beta):
    lbd = np.exp(X @ beta)
    a = y * np.log(lbd)  # ln(lbd^y)
    b = - np.log(factorial(y))  # ln(y!)
    c = - lbd  # ln(e^-lbd)
    return -(a + b + c).sum()

res_pos = minimize(log_likelihood_poisson, x0=init_beta)
res_pos

      fun: 3.37835550522458
 hess_inv: array([[29.62135324, -4.51187731, -4.23965362],
       [-4.51187731,  0.72888415,  0.61098797],
       [-4.23965362,  0.61098797,  0.66601047]])
      jac: array([ 7.15255737e-07,  4.85777855e-06, -4.17232513e-07])
  message: 'Optimization terminated successfully.'
     nfev: 156
      nit: 32
     njev: 39
   status: 0
  success: True
        x: array([-6.07848311,  0.93340263,  0.84329618])